# Test SQLite file locking

In [ ]:
import pytest
import lamindb_setup as ln_setup
from lamindb_setup.core.cloud_sqlite_locker import Locker, InstanceLockedException
from lamindb_setup.core.upath import UPath

In [ ]:
ln_setup.login("testuser2")

In [ ]:
root = UPath("s3://lamindb-ci/test-load-lock", cache_regions=True)

In [ ]:
exclusion = root / ".lamindb/_exclusion/"
# if cache is present, then .exists() for intermediate keys return False
# bug in s3fs
exclusion.fs.invalidate_cache()
if exclusion.exists():
    exclusion.rmdir()

In [ ]:
assert ln_setup.settings.user.handle == "testuser2"

In [ ]:
ln_setup.init(storage="s3://lamindb-ci/test-load-lock", name="test-load-lock")
instance_id = ln_setup.settings.instance._id

assert (
    ln_setup.settings.instance._cloud_sqlite_locker
    is ln_setup.settings.instance._cloud_sqlite_locker
)

ln_setup.close()

In [ ]:
# lock with some random id
locker = Locker("randuseridtt", storage_root=root, instance_id=instance_id)

In [ ]:
locker.lock()

In [ ]:
# with pytest.raises(InstanceLockedException):
#    ln_setup.connect("test-load-lock"), would like to test this, but need another python session

In [ ]:
locker.unlock()

In [ ]:
# ln_setup.connect("test-load-lock")
# ln_setup.close()

In [ ]:
# lock through testuser1, who has user id "DzTjkKse"
locker = Locker("DzTjkKse", storage_root=root, instance_id=instance_id)

In [ ]:
locker.lock()

In [ ]:
# with pytest.raises(InstanceLockedException) as error:
#     ln_setup.connect("test-load-lock")

# assert (
#     "InstanceLockedException: Cannot load the instance, it is locked by 'testuser1'"
#     " (uid: 'DzTjkKse', name: 'Test User1')."
#     in error.exconly()
# )

In [ ]:
locker.unlock()

In [ ]:
# ln_setup.connect("test-load-lock")

In [ ]:
# test ignore_prev_locker=True in unlock_cloud_sqlite_upon_exception
# i.e. test that the locker doesn't unlock if the locker hasn't changed during the function execution
# with pytest.raises(RuntimeError):
#     ln_setup.connect("test-load-lock")

# assert ln_setup.settings.instance._cloud_sqlite_locker._has_lock

In [ ]:
# ln_setup.close()

In [ ]:
# purely technical varibale to test failed load after locking
# ln_setup._connect_instance._TEST_FAILED_LOAD = True

# with pytest.raises(RuntimeError):
#     ln_setup.connect("test-load-lock")

# assert ln_setup.core.cloud_sqlite_locker._locker._has_lock is None

# ln_setup._connect_instance._TEST_FAILED_LOAD = False

In [ ]:
# ln_setup.connect("test-load-lock", _test=True)

In [ ]:
ln_setup.delete("test-load-lock", force=True)